## import libraries

### pip install

In [12]:
#pip install
!pip install faiss-gpu --quiet
!pip install boto3 --quiet
!pip install langchain-aws --quiet
!pip install langchain-community boto3 --quiet
!pip install boto3 requests requests-aws4auth --quiet
!pip install pymongo --quiet
!pip install google-search-results sagemaker --quiet
!pip install boto3 nltk --quiet
!pip install wikipedia --quiet
!pip install tavily-python --quiet
!pip install --upgrade langchain sympy numexpr --quiet
!pip install langchain-experimental --quiet
!pip install -U langgraph langchain_community langchain_anthropic langchain_experimental --quiet
!pip install langchain-redis langchain-openai redis --quiet
#!docker run -d -p 6379:6379 redis:latest --quiet

### libraries

In [13]:
import boto3
import os
from langchain import hub

#chat models
from langchain_aws import ChatBedrock
from langchain_aws import ChatBedrockConverse

#LLMs
from langchain_aws import BedrockLLM
from langchain_community.llms import AmazonAPIGateway
from langchain_aws import SagemakerEndpoint

#prompts
from langchain.prompts import PromptTemplate

#embedding models
from langchain_aws import BedrockEmbeddings
# from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import ContentHandlerBase

#document loaders
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
from langchain_community.document_loaders import AmazonTextractPDFLoader

#vector stores
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_community.vectorstores import DocumentDBVectorSearch

from langchain_aws.vectorstores.inmemorydb import InMemoryVectorStore

#retrievers
from langchain_aws import AmazonKendraRetriever
from langchain_aws import AmazonKnowledgeBasesRetriever

#memory
from langchain_community.chat_message_histories import DynamoDBChatMessageHistory
from langchain_community.chat_message_histories.redis import RedisChatMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables import RunnablePassthrough

#graphs
from langchain_community.graphs import NeptuneGraph
from langchain_community.graphs import NeptuneAnalyticsGraph
from langchain_community.chains.graph_qa.neptune_cypher import NeptuneOpenCypherQAChain

from langchain_community.graphs import NeptuneRdfGraph
from langchain_community.chains.graph_qa.neptune_sparql import NeptuneSparqlQAChain

#callbacks
from langchain_community.callbacks.bedrock_anthropic_callback import BedrockAnthropicTokenUsageCallbackHandler
from langchain_community.callbacks import SageMakerCallbackHandler

#chains
# from langchain_experimental.comprehend_moderation import AmazonComprehendModerationChain

#tavily
from tavily import TavilyClient
from langchain_community.tools.tavily_search import TavilySearchResults

#tools
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.tools import BaseTool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader, CSVLoader
from langchain.tools.retriever import create_retriever_tool
from langchain_core.tools import Tool
from langchain_experimental.utilities import PythonREPL

#agent
from langchain.agents import create_tool_calling_agent, AgentType
from langchain.agents import AgentExecutor
from langchain.chat_models import ChatOpenAI

USER_AGENT environment variable not set, consider setting it to identify your requests.


## setup agent

### setup llm

In [14]:
model_parameter = {"temperature": 0.9, "top_p": .5, "max_tokens_to_sample": 200}
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"
bedrock_llm = ChatBedrock(
    model_id=modelId,
    client=boto3.client('bedrock'),
    model_kwargs=model_parameter, 
    beta_use_converse_api=True
)

In [15]:
embeddings = BedrockEmbeddings(
    client=boto3.client('bedrock-runtime', region_name="us-east-1"),
    model_id = "amazon.titan-embed-text-v2:0"
)

### initiate tools

In [16]:
os.environ["TAVILY_API_KEY"] = "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"
search = TavilySearchResults()

In [17]:
prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],  # Include both input and agent_scratchpad
    template="""
    
If the input is to do with retail or an item to buy, do the following:

Use the retriever tool to answer any queries related to internal data files. 
Only use external tools (like Wikipedia) if no relevant information is found internally.
When asked about a specific item output the answers to the following queries: 

Should this item be ordered? 
Find out what date it is currently using date time tool. Convert the month and year into strings and use these for the rest of the answer.
Use recent stock statistics to see if the stocks for this month are low compared to how much was used last month.
Use the statistics from last year to predict how much of each item will be needed for the rest of this month and next month.
Use what month it is at the time the call is being made and whether any important holidays eg christmas or summer will be coming up and use this information in your decision.
a) how much should be ordered for this month and next month
b) how much should be ordered in the next few months


What brand?
Use the customers reviews to decide this. 

If this item should be ordered, how much of this item should be ordered?
Use the repl tool to calculate these numbers.

In your final output:
1. Explain your reasoning for whether an item should be ordered and how much.
Don't explain your reasoning for
Explain the maths you used.
2. Draft an email to an imaginary supplier of this brand to order the number of the item you think should be ordered, signing off as Reply Auto Replenishment.

Else:
Answer the question appropriately using multiple tools to verify your answer

Query: {input}

{agent_scratchpad}
"""
)


In [18]:
# Define the path to the directory containing CSV files
data_files_directory = "data_files"

# Use DirectoryLoader to load all CSV files
loader = DirectoryLoader(
    data_files_directory,
    glob="**/*.csv",  # Match all CSV files in the directory (including subdirectories)
    loader_cls=CSVLoader  # Use CSVLoader for parsing the files
)

In [19]:
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "Retriever_tool",
    "First search internal data files to answer any queries. If there is no useful information then use other tools.",
)

In [20]:
#datetime tool
from datetime import datetime
datetime_tool = Tool(
    name="Datetime",
    func=lambda x: datetime.now().isoformat(),
    description="Returns the current datetime",
)


In [21]:
#loading in tools

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Use this to execute python commands and calculations.",
    func=python_repl.run,
)
wikipedia_tool = load_tools(["wikipedia"], llm=bedrock_llm)



In [56]:
def write_email(recipient,my_name,subject,text,html):
    data = {
        'FromEmail': 'sneville1313@gmail.com',
        'FromName': my_name,
        'Subject': subject,
        'Text-part': text,
        'Html-part': html,
        'Recipients': [{'Email': recipient}]
}
    result = mailjet.send.create(data=data)
    print(result.status_code)
    print(result.json())
    
writing_email_tool = Tool(
    name="Send Email",
    func=lambda args: write_email(
        recipient=args.get("recipient"),
        my_name=args.get("my_name"),
        subject=args.get("subject"),
        text=args.get("text"),
        html=args.get("html")
    ),
    description="Sends an email to a specified recipient. Parameters: recipient (str), my_name (str), subject (str), text (str), html (str)."
)

In [46]:
tools = [datetime_tool,retriever_tool,search,repl_tool,wikipedia_tool[0],writing_email_tool]

## running agent

### general agent

In [22]:
def run_agent(prompt,agent_input,raw_list,verbose_setting):
    agent = create_tool_calling_agent(bedrock_llm, tools, prompt)
    agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=verbose_setting,handle_parsing_errors=True)
    response = agent_executor.invoke({"input": agent_input})
    raw_output = response['output'][0]['text']
    raw_list.append(raw_output)

In [23]:
raw_list = []
run_agent(prompt,'christmas pudding',raw_list,True)
run_agent(prompt,'suncream',raw_list,False)
for i in range(len(raw_list)):
    print('-'*20)
    print(raw_list[i])
    print('\n')



> Entering new AgentExecutor chain...

Invoking: `Retriever_tool` with `{'query': 'christmas pudding'}`
responded: [{'type': 'tool_use', 'name': 'Retriever_tool', 'id': 'tooluse_AJ0H4CQHRou0qOTEtTRI1A', 'index': 0, 'input': '{"query": "christmas pudding"}'}]

Item: Christmas Pudding
Jan: 0
Feb: 0
Mar: 0
Apr: 0
May: 0
Jun: 0
Jul: 0
Aug: 0
Sep: 0
Oct: 0
Nov: 0
Dec: 20
Brand_A_Jan: 0
Brand_A_Feb: 0
Brand_A_Mar: 0
Brand_A_Apr: 0
Brand_A_May: 0
Brand_A_Jun: 0
Brand_A_Jul: 0
Brand_A_Aug: 0
Brand_A_Sep: 0
Brand_A_Oct: 0
Brand_A_Nov: 0
Brand_A_Dec: 12
Brand_B_Jan: 0
Brand_B_Feb: 0
Brand_B_Mar: 0
Brand_B_Apr: 0
Brand_B_May: 0
Brand_B_Jun: 0
Brand_B_Jul: 0
Brand_B_Aug: 0
Brand_B_Sep: 0
Brand_B_Oct: 0
Brand_B_Nov: 0
Brand_B_Dec: 8
Brand_C_Jan: 0
Brand_C_Feb: 0
Brand_C_Mar: 0
Brand_C_Apr: 0
Brand_C_May: 0
Brand_C_Jun: 0
Brand_C_Jul: 0
Brand_C_Aug: 0
Brand_C_Sep: 0
Brand_C_Oct: 0
Brand_C_Nov: 0
Brand_C_Dec: 0

Item: Christmas Pudding
Used_Last_Month: 5
Left_This_Month: 2
Brand_A_Used_Last_Month: 

Python REPL can execute arbitrary code. Use with caution.



Invoking: `python_repl` with `{'Arg1': 'dec_order = 12\njan_order = 3\ntotal_order = dec_order + jan_order\nprint(f"Order {total_order} units of Brand B Christmas pudding for Dec 2024 and Jan 2025")'}`
responded: [{'type': 'text', 'text': "Here is my analysis and recommendation for ordering Christmas pudding:\n\nShould this item be ordered?\nYes, Christmas pudding should be ordered based on the sales data showing demand in December.\n\nFind out what date it is currently using datetime tool. Convert the month and year into strings and use these for the rest of the answer.\nThe current date is 2024-12-12. The month is December and the year is 2024.\n\nUse recent stock statistics to see if the stocks for this month are low compared to how much was used last month. \nFor December 2023 (last month), 5 units of Christmas pudding were sold, with 2 units left over this month. So stocks are relatively low compared to last month's usage.\n\nUse the statistics from last year to predict how much 

## just output email

In [57]:
email_prompt = PromptTemplate(
    input_variables=["input", "agent_scratchpad"],  # Include both input and agent_scratchpad
    template="""
Use the retriever tool to answer any queries related to internal data files. 
Only use external tools (like Wikipedia) if no relevant information is found internally.
When asked about a specific item output the answers to the following queries: 

Should this item be ordered? 
Find out what date it is currently using date time tool. Convert the month and year into strings and use these for the rest of the answer.
Use recent stock statistics to see if the stocks for this month are low compared to how much was used last month.
Use the statistics from last year to predict how much of each item will be needed for the rest of this month and next month.
Use what month it is at the time the call is being made and whether any important holidays eg christmas or summer will be coming up and use this information in your decision.
a) how much should be ordered for this month and next month
b) how much should be ordered in the next few months

What brand?
Use the customers reviews to decide this. 

If this item should be ordered, how much of this item should be ordered?
Use the repl tool to calculate these numbers.

Draft an email to an imaginary supplier of this brand to order the number of the item you think should be ordered, signing off as Reply Auto Replenishment.
If you decide that an item should be ordered, use the email writing tool to send the appropriate email.
The parameters to the function writing_email should be:
recipient='lucaswong631@gmail.com
my_name= [you as a model decide name of sender]
subject= [you as a model decide subject of email]
text= [email you drafted]
html= [any html you want to add in, if not set it as '']
The only thing you should output is the email and 'Email sent', or 'No need to order more of [item]'.
If you decide there is no need to order more of an item, explain why, but if you send an email do not output anything else back.

Query: {input}

{agent_scratchpad}
"""
)

### sending emails

In [25]:
!pip install mailjet_rest

In [26]:
from mailjet_rest import Client
import os

os.environ['MJ_APIKEY_PUBLIC'] = '3ca28dfede990c7e1df73da9f7303f06'
os.environ['MJ_APIKEY_PRIVATE'] = '9c4976205a677cf6c975dc22ae43656e'

# Access them using os.environ
api_key = os.environ['MJ_APIKEY_PUBLIC']
api_secret = os.environ['MJ_APIKEY_PRIVATE']

# Initialize Mailjet client
mailjet = Client(auth=(api_key, api_secret))


In [30]:
raw_list=[]
run_agent(email_prompt,'tooth paste',raw_list,False)

In [32]:
lucas_email = 'lucaswong631@gmail.com'
write_email(lucas_email,'Data Reply Autoreplenishment','Incoming supply request',raw_list[0],'')

200
{'Sent': [{'Email': 'lucaswong631@gmail.com', 'MessageID': 288230403600958000, 'MessageUUID': '27a0995f-054e-446c-825f-f161883ab4a9'}]}


In [38]:
run_agent(email_prompt,'bread',raw_list,False) # DOES NOT WORk !!! not sure why

ValidationException: An error occurred (ValidationException) when calling the ConverseStream operation: 1 validation error detected: Value 'Send Email' at 'toolConfig.tools.6.member.toolSpec.name' failed to satisfy constraint: Member must satisfy regular expression pattern: [a-zA-Z][a-zA-Z0-9_]*